# Issue53

* Timelink Python package version: 1.1.13
* Python version: 3.10, 3.11
* Operating System: MacOs

### Description

it is not always possible to get the ORM corresponding to a group because a group
can extend another one and not have its own mapping. However internally Timelink
knows the model to use, but the end user need to inspect the structure file to figure out.

In str
```
pars name=lugar; source=geoentity
```

in python
```python
db.get_model("lugar") # returns none
```

Or make get_model smarter.

In [1]:
# NBVAL_IGNORE_OUTPUT
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook(db_name='issue53', kleio_image='kleio-server', kleio_version='12.6.575')
tlnb.print_info()

Timelink version: 1.1.13
Project name: test-project
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project
Database type: sqlite
Database name: issue53
Kleio image: kleio-server
Kleio server token: 98Sct...
Kleio server URL: http://127.0.0.1:8088
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project
Kleio server container: adoring_germain
Kleio version requested: 12.6.575
Kleio server version: 12.6.575 (2024-09-06 14:08:58)
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project/database/sqlite
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=test-project, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project, db_type=sqlite, db_name=issue53, kleio_image=kleio-server, kleio_version=12.6.575, postgres_image=postgres, postgres_version=latest)


## import a file with groups with no direct mapping

In [2]:
# tlnb.kleio_server.translate("sources/cronologias/crono-avulsos.cli")


([{'job': 28, 'sources': ['sources/cronologias/crono-avulsos.cli']}], None)

In [3]:
# NBVAL_IGNORE_OUTPUT

import logging
logging.basicConfig(level=logging.INFO)

path = "sources/cronologias/"
tlnb.update_from_sources(path=path)
# get the import status
import_status = tlnb.get_import_status(path=path)
# list those with import_error > 0 or error > 0
errors = import_status.query("import_errors > 0 or errors > 0")
if len(errors) > 0:
    print(errors)

INFO:root:Importing sources/cronologias/crono-avulsos.cli


In [8]:
# NBVAL_IGNORE_OUTPUT

kleio_files = tlnb.get_kleio_files(path=path)
kleio_files

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,sources/cronologias/crono-avulsos.cli,crono-avulsos.cli,2024-09-11 03:17:19.547244+00:00,V,2024-09-11 03:17:00+00:00,0,0,I,0,0,No errors,No warnings,2024-09-11 03:17:20.646885,/rest/reports/sources/cronologias/crono-avulso...,/rest/exports/sources/cronologias/crono-avulso...


In [9]:
from sqlalchemy import select
from sqlalchemy.orm import aliased

eventos, lugares = tlnb.db.get_model(["crono","lugar"])

stmt = (select(eventos, lugares)
        .join(lugares,
              eventos.id == lugares.inside)
        .order_by(lugares.name, eventos.the_date))

with tlnb.db.session() as session:
    results = session.execute(stmt).all()
    for evento, lugar in results:
        print(f"{lugar.name:16} {evento.the_date} {evento.description}")

NoInspectionAvailable: No inspection system is available for object of type <class 'NoneType'>